In [ ]:
import re
from functools import cache

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
lines_split = [l.split() for l in lines]
records = [
    (r[0], [int(n) for n in re.findall(r"(-?[\d]+)", r[1])]) for r in lines_split
]
records[:5]

In [ ]:
# The general idea is considering the search space a tree
# Each question mark means two possible children
# A recursive depth first search is applied to search through all possible paths
# Whenever it is not possible anymore to finish, it backtracks

In [ ]:
def search_configurations(
    configuration,
    counts,
    damaged_streak,
    damaged_count,
    total_damaged,
    conf_acc,
    debug=False,
) -> int:
    if debug:
        print(
            f"configuration={configuration},counts={counts},damaged_streak={damaged_streak},damaged_count={damaged_count},total_damaged={total_damaged},conf_acc={conf_acc}"
        )

    if configuration == "":
        # It ended on a leaf node and all required numbers match up
        if damaged_count == total_damaged and (
            len(counts) == 0 or damaged_streak == counts[0]
        ):
            if debug:
                print(f"Valid conf: {conf_acc}")
            return 1
        else:
            # Numbers to not match on a leaf -> invalid path
            return 0

    if total_damaged > damaged_count + len(configuration):
        # Not enough damaged components can be placed in remaining child nodes to match total damaged components required
        return 0

    current_node = configuration[0]
    if current_node == "?":
        # In case of question mark, explore both possible child paths
        if not counts:
            # If counts is empty, no more damaged components are needed, only explore "." children
            return search_configurations(
                "." + configuration[1:],
                counts,
                damaged_streak,
                damaged_count,
                total_damaged,
                conf_acc,
                debug,
            )

        return search_configurations(
            "." + configuration[1:],
            counts,
            damaged_streak,
            damaged_count,
            total_damaged,
            conf_acc,
            debug,
        ) + search_configurations(
            "#" + configuration[1:],
            counts,
            damaged_streak,
            damaged_count,
            total_damaged,
            conf_acc,
            debug,
        )

    if current_node == ".":
        if damaged_streak > 0 and counts and damaged_streak == counts[0]:
            # Streak of damaged components and it matches required count -> count is done, streak reset
            counts = counts[1:]
            damaged_streak = 0

        if damaged_streak > 0 and counts and damaged_streak != counts[0]:
            # Streak of damaged components but doesnt match required count at this point -> abort
            return 0

        return search_configurations(
            configuration[1:],
            counts,
            damaged_streak,
            damaged_count,
            total_damaged,
            conf_acc + ".",
            debug,
        )

    if current_node == "#":
        if damaged_count > total_damaged:
            # Damaged comp count now higher than total count required -> abort
            return 0

        if counts and damaged_streak >= counts[0]:
            # Streak higher than what current count requires -> abort
            return 0

        return search_configurations(
            configuration[1:],
            counts,
            damaged_streak + 1,
            damaged_count + 1,
            total_damaged,
            conf_acc + "#",
            debug,
        )

    return -1


search_configurations("????????", [1, 3], 0, 0, 4, "", debug=True)

In [ ]:
res_list = []
for r in records:
    configs = search_configurations(r[0], r[1], 0, 0, sum(r[1]), "")
    res_list.append(configs)
sum(res_list)

part 2

In [ ]:
# Unfortunately part 1 is really slow on part 2 input

# However, there are a _lot_ of subtrees that are equal in all directions if you don't consider the actual path taken there
# so removing the path recording (which is nice for debugging but not needed for the result) and making the counts list a tuple to be hashable
# enables the possibility of adding a results cache
# and instead of taking forever, it now takes under a second

In [ ]:
records_2 = records.copy()
records_2 = [("?".join([r[0]] * 5), r[1] * 5) for r in records_2]
records_2[0]

In [ ]:
@cache
def search_configurations_c(
    configuration, counts, damaged_streak, damaged_count, total_damaged, debug=False
) -> int:
    if configuration == "":
        if damaged_count == total_damaged and (
            len(counts) == 0 or damaged_streak == counts[0]
        ):
            return 1
        else:
            return 0

    if total_damaged > damaged_count + len(configuration):
        return 0

    current_node = configuration[0]
    if current_node == "?":
        if not counts:
            return search_configurations_c(
                "." + configuration[1:],
                tuple(counts),
                damaged_streak,
                damaged_count,
                total_damaged,
                debug,
            )
        return search_configurations_c(
            "." + configuration[1:],
            counts,
            damaged_streak,
            damaged_count,
            total_damaged,
            debug,
        ) + search_configurations_c(
            "#" + configuration[1:],
            tuple(counts),
            damaged_streak,
            damaged_count,
            total_damaged,
            debug,
        )

    if current_node == ".":
        if damaged_streak > 0 and counts and damaged_streak == counts[0]:
            counts = counts[1:]
            damaged_streak = 0

        if damaged_streak > 0 and counts and damaged_streak != counts[0]:
            return 0

        return search_configurations_c(
            configuration[1:],
            tuple(counts),
            damaged_streak,
            damaged_count,
            total_damaged,
            debug,
        )

    if current_node == "#":
        if damaged_count > total_damaged:
            return 0

        if counts and damaged_streak >= counts[0]:
            return 0

        return search_configurations_c(
            configuration[1:],
            tuple(counts),
            damaged_streak + 1,
            damaged_count + 1,
            total_damaged,
            debug,
        )

    return -1


search_configurations_c("????????", tuple([1, 3]), 0, 0, 4, debug=True)

In [ ]:
res_list = []
for r in records_2:
    configs = search_configurations_c(r[0], tuple(r[1]), 0, 0, sum(r[1]))
    res_list.append(configs)
sum(res_list)